### HVS Data Processing Pipeline
Generating 6D Phase Space for select HVS 

In [1]:
import pandas as pd
from astroquery.gaia import Gaia

print("Imported libraries successfully...")

Imported libraries successfully...


In [19]:
# Load CSV into Pandas dataframe
df = pd.read_csv('data/source_data.csv', delimiter=',')
pd.set_option('display.width', 160)
print(df)
print(df.columns)

    HVS            source_id          vhelio          pmra         pmdec          dhelio   pMW  pLMC  log_z_MW
0     1   577294697514301440   831.10 ± 5.70  -0.60 ± 0.60  -0.47 ± 0.39  102.24 ± 14.60  0.72  0.12    -0.490
1     4   699811079173836928   600.90 ± 6.20  -0.20 ± 0.26  -0.60 ± 0.19    63.80 ± 9.70  0.92  0.01    -2.600
2     5  1069326945513133952   545.50 ± 4.30   0.00 ± 0.08  -0.99 ± 0.11    44.20 ± 5.09  0.00  0.00       NaN
3     6  3867267443277880320   609.40 ± 6.80   0.12 ± 0.30   0.12 ± 0.23    55.36 ± 6.88  0.00  0.46     7.000
4     7  3799146650623432704   526.90 ± 3.00  -0.09 ± 0.18   0.02 ± 0.13    52.17 ± 6.25  0.00  0.49    27.000
5     8   633599760258827776   499.30 ± 2.90  -0.88 ± 0.16  -0.28 ± 0.14    53.19 ± 9.80  0.00  0.00   -53.000
6     9  3830584196322129920   616.80 ± 5.10   0.26 ± 0.43  -0.81 ± 0.65   74.10 ± 11.60  0.44  0.77    -0.130
7    10  3926757653770374272   467.90 ± 5.60  -1.09 ± 0.45  -0.99 ± 0.21    51.76 ± 5.72  0.57  0.80    -0.380
8

In [20]:
# Clean columns with +- format, drop unneeded columns (pMW, pLMC, log_z_MW)

drop_cols = ['pMW', 'pLMC', 'log_z_MW']
split_cols = ['vhelio', 'pmra', 'pmdec', 'dhelio']

for drop in drop_cols:
    df = df.drop(drop, axis=1)
print(df)

    HVS            source_id          vhelio          pmra         pmdec          dhelio
0     1   577294697514301440   831.10 ± 5.70  -0.60 ± 0.60  -0.47 ± 0.39  102.24 ± 14.60
1     4   699811079173836928   600.90 ± 6.20  -0.20 ± 0.26  -0.60 ± 0.19    63.80 ± 9.70
2     5  1069326945513133952   545.50 ± 4.30   0.00 ± 0.08  -0.99 ± 0.11    44.20 ± 5.09
3     6  3867267443277880320   609.40 ± 6.80   0.12 ± 0.30   0.12 ± 0.23    55.36 ± 6.88
4     7  3799146650623432704   526.90 ± 3.00  -0.09 ± 0.18   0.02 ± 0.13    52.17 ± 6.25
5     8   633599760258827776   499.30 ± 2.90  -0.88 ± 0.16  -0.28 ± 0.14    53.19 ± 9.80
6     9  3830584196322129920   616.80 ± 5.10   0.26 ± 0.43  -0.81 ± 0.65   74.10 ± 11.60
7    10  3926757653770374272   467.90 ± 5.60  -1.09 ± 0.45  -0.99 ± 0.21    51.76 ± 5.72
8    12  3809777626689513216   552.20 ± 6.60   0.93 ± 0.88  -0.19 ± 0.58    64.83 ± 8.36
9    13  3804790100211231104   569.30 ± 6.10   0.07 ± 0.79  -0.20 ± 0.63  105.58 ± 19.45
10   14  385927533377